In [172]:
import numpy as np
import pandas as pd

# Data
class_10_percentage = [90,92,85,78,96,98,84,72,95,98,87,83,78,75,91,95,94,88,93,81]
class_12_percentage = [85,89,87,81,94,95,88,79,91,95,91,87,84,77,88,91,92,84,90,88]
jee_main_percentile = [88,91,92,82,96,98,91,69,93,96,94,92,89,82,92,94,97,77,87,77]

data = pd.DataFrame({'Class10Percentage': class_10_percentage,
                     'Class12Percentage': class_12_percentage,
                     'JEE_MainPercentile': jee_main_percentile})
data

,Class10Percentage,Class12Percentage,JEE_MainPercentile
0,90,85,88
1,92,89,91
2,85,87,92
3,78,81,82
4,96,94,96
5,98,95,98
6,84,88,91
7,72,79,69
8,95,91,93
9,98,95,96


In [173]:
# Normalize data
data_normalized = (data - data.mean()) / data.std()
data_normalized

,Class10Percentage,Class12Percentage,JEE_MainPercentile
0,0.298888,-0.559882,-0.109998
1,0.553261,0.239949,0.278230
2,-0.337044,-0.159966,0.407639
3,-1.227350,-1.359714,-0.886453
4,1.062007,1.239739,0.925275
5,1.316380,1.439697,1.184094
6,-0.464231,0.039992,0.278230
7,-1.990469,-1.759630,-2.568772
8,0.934821,0.639865,0.537048
9,1.316380,1.439697,0.925275


In [174]:
X = data_normalized[['Class10Percentage', 'Class12Percentage']].values
y = data_normalized['JEE_MainPercentile'].values.reshape(-1, 1)

# Define hyperparameters
input_size = X.shape[1]
output_size = 1
hidden_size = 32
learning_rate = 0.021
epochs = 1000

# Initialize weights and biases
np.random.seed(42)
weights_input_hidden = 0.01*np.random.randn(input_size, hidden_size)
bias_hidden = np.zeros((1, hidden_size))
weights_hidden_output = 0.01*np.random.randn(hidden_size, output_size)
bias_output = np.zeros((1, output_size))

In [175]:
# Define activation function (ReLU)
def relu(x):
    return np.maximum(0, x)

# Define derivative of ReLU
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [176]:
for epoch in range(epochs):
    # Forward pass
    hidden_layer_input = np.dot(X, weights_input_hidden) + bias_hidden
    hidden_layer_output = relu(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    predicted_output = output_layer_input
    
    # Compute loss (Mean Squared Error)
    loss = np.mean((predicted_output - y)**2)
    
    # Backward pass
    output_error = predicted_output - y
    output_delta = output_error
    hidden_error = np.dot(output_delta, weights_hidden_output.T)
    hidden_delta = hidden_error * relu_derivative(hidden_layer_input)
    
    # Update weights and biases
    weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, output_delta)
    bias_output -= learning_rate * np.sum(output_delta, axis=0, keepdims=True)
    weights_input_hidden -= learning_rate * np.dot(X.T, hidden_delta)
    bias_hidden -= learning_rate * np.sum(hidden_delta, axis=0, keepdims=True)
    
    if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Loss: {loss}')

Epoch: 0, Loss: 0.9495894331260665
Epoch: 100, Loss: 0.3474855312217392
Epoch: 200, Loss: 0.343262046099846
Epoch: 300, Loss: 0.3254916747850405
Epoch: 400, Loss: 0.23503837092652144
Epoch: 500, Loss: 0.15024053442148513
Epoch: 600, Loss: 0.1434985294564438
Epoch: 700, Loss: 0.1541734301197866
Epoch: 800, Loss: 0.14804187962979332


Epoch: 900, Loss: 0.1443894223748303
Epoch: 1000, Loss: 0.14219037076086222
Epoch: 1100, Loss: 0.13613790915525353


In [177]:
def predict(X):
    hidden_layer_input = np.dot(X, weights_input_hidden) + bias_hidden
    hidden_layer_output = relu(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    return output_layer_input

In [178]:
predicted_percentile_normalized = predict(X)
predicted_percentile = (predicted_percentile_normalized * data['JEE_MainPercentile'].std()) + data['JEE_MainPercentile'].mean()

# Display predictions
predictions = pd.DataFrame({'Actual': data['JEE_MainPercentile'], 'Predicted': predicted_percentile.flatten()})
print(predictions)

    Actual  Predicted
0       88  83.334952
1       91  90.944594
2       92  90.364387
3       82  84.392331
4       96  96.431810
5       98  97.177366
6       91  91.644250
7       69  66.103684
8       93  92.712590
9       96  97.177366
10      94  93.747060
11      92  90.950254
12      89  82.662642
13      82  78.386625
14      92  89.922157
15      94  92.712590
16      97  94.347677
17      77  82.009479
18      87  91.967034
19      77  77.434885


In [179]:
mse = np.mean((data["JEE_MainPercentile"] - predicted_percentile.flatten())**2)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 7.932477365771476


In [180]:
mae = np.mean(np.abs( data["JEE_MainPercentile"] - predicted_percentile.flatten()))
print(f'Mean Absolute Error: {mae}')


Mean Absolute Error: 2.1345288191553253


In [181]:
ss_total = np.sum((data["JEE_MainPercentile"]  - np.mean(data["JEE_MainPercentile"] ))**2)
ss_residual = np.sum((data["JEE_MainPercentile"]  - predicted_percentile.flatten())**2)
r2 = 1 - (ss_residual / ss_total)
print(f'R-squared: {r2}')

R-squared: 0.8601652220568248
